In [2]:
filename = 'BenchmarkingZeroShot/topic/train_pu_half_v0.txt'

with open(filename) as f:
    data = f.readlines()

In [3]:

data[0].strip()
line_co=0
exam_co = 0
examples=[]
seen_types = set()

for row in data:
    line = row.strip().split('\t')
    if len(line) == 2: # label_id, text
        type_index = int(line[0])
        seen_types.add(type_index)

seen_types


{0, 2, 4, 6, 8}

In [4]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


In [8]:
type2hypothesis = {
0: ['it is related with society or culture', 'this text  describes something about an extended social group having a distinctive cultural and economic organization or a particular society at a particular time and place'],
1:['it is related with science or mathematics', 'this text  describes something about a particular branch of scientific knowledge or a science (or group of related sciences) dealing with the logic of quantity and shape and arrangement'],
2: ['it is related with health', 'this text  describes something about a healthy state of wellbeing free from disease'],
3: ['it is related with education or reference', 'this text  describes something about the activities of educating or instructing or activities that impart knowledge or skill or an indicator that orients you generally'],
4: ['it is related with computers or Internet', 'this text  describes something about a machine for performing calculations automatically or a computer network consisting of a worldwide network of computer networks that use the TCP/IP network protocols to facilitate data transmission and exchange'],
5: ['it is related with sports', 'this text  describes something about an active diversion requiring physical exertion and competition'],
6: ['it is related with business or finance', 'this text  describes something about a commercial or industrial enterprise and the people who constitute it or the commercial activity of providing funds and capital'],
7: ['it is related with entertainment or music', 'this text  describes something about an activity that is diverting and that holds the attention or an artistic form of auditory communication incorporating instrumental or vocal tones in a structured and continuous manner'],
8: ['it is related with family or relationships', 'this text  describes something about a social unit living together, primary social group; parents and children or a relation between people'],
9: ['it is related with politics or government', 'this text  describes something about social relations involving intrigue to gain authority or power or the organization that is the governing authority of a political unit']}


In [5]:
from collections import defaultdict
type2hypothesis = {
0: ['it is related with society or culture', 'this text  describes something about an extended social group having a distinctive cultural and economic organization or a particular society at a particular time and place'],
1:['it is related with science or mathematics', 'this text  describes something about a particular branch of scientific knowledge or a science (or group of related sciences) dealing with the logic of quantity and shape and arrangement'],
2: ['it is related with health', 'this text  describes something about a healthy state of wellbeing free from disease'],
3: ['it is related with education or reference', 'this text  describes something about the activities of educating or instructing or activities that impart knowledge or skill or an indicator that orients you generally'],
4: ['it is related with computers or Internet', 'this text  describes something about a machine for performing calculations automatically or a computer network consisting of a worldwide network of computer networks that use the TCP/IP network protocols to facilitate data transmission and exchange'],
5: ['it is related with sports', 'this text  describes something about an active diversion requiring physical exertion and competition'],
6: ['it is related with business or finance', 'this text  describes something about a commercial or industrial enterprise and the people who constitute it or the commercial activity of providing funds and capital'],
7: ['it is related with entertainment or music', 'this text  describes something about an activity that is diverting and that holds the attention or an artistic form of auditory communication incorporating instrumental or vocal tones in a structured and continuous manner'],
8: ['it is related with family or relationships', 'this text  describes something about a social unit living together, primary social group; parents and children or a relation between people'],
9: ['it is related with politics or government', 'this text  describes something about social relations involving intrigue to gain authority or power or the organization that is the governing authority of a political unit']}

type_load_size = defaultdict(int)

for row in data:
    line = row.strip().split('\t')
    if len(line) == 2: # label_id, text
        type_index = int(line[0])
        if type_load_size.get(type_index, 0) < 10000:
            for i in range(10):
                hypo_list = type2hypothesis.get(i)
                if i == type_index:
                    '''pos pair'''
                    for hypo in hypo_list:
                        guid = "train-"+str(exam_co)
                        text_a = line[1]
                        text_b = hypo
                        label = 'entailment' #if line[0] == '1' else 'not_entailment'
                        examples.append(
                            InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
                        exam_co+=1
                elif i in seen_types:
                    '''neg pair'''
                    for hypo in hypo_list:
                        guid = "train-"+str(exam_co)
                        text_a = line[1]
                        text_b = hypo
                        label = 'not_entailment' #if line[0] == '1' else 'not_entailment'
                        examples.append(
                            InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
                        exam_co+=1
            line_co+=1
            if line_co % 10000 == 0:
                print('loading training size:', line_co)

            type_load_size[type_index]+=1
        else:
            continue

loading training size: 10000
loading training size: 20000
loading training size: 30000
loading training size: 40000
loading training size: 50000


In [1]:
from collections import defaultdict
a = defaultdict(list)
a['2'].append(1)
a

defaultdict(list, {'2': [1]})

In [6]:
exam_co

500000

In [7]:
filename = 'BenchmarkingZeroShot/topic/test.txt'
with open(filename) as f:
    data = f.readlines()
line_co = 0 
exam_co = 0
examples = []

hypo_seen_str_indicator=[]
hypo_2_type_index=[]
for i in range(10):
    hypo_list = type2hypothesis.get(i)
    for hypo in hypo_list:
        hypo_2_type_index.append(i) # this hypo is for type i
        if i in seen_types:
            hypo_seen_str_indicator.append('seen')# this hypo is for a seen type
        else:
            hypo_seen_str_indicator.append('unseen')
gold_label_list = []
for row in data:
    line=row.strip().split('\t')
    if len(line)==2: # label_id, text

        type_index =  int(line[0])
        gold_label_list.append(type_index)
        for i in range(10):
            hypo_list = type2hypothesis.get(i)
            if i == type_index:
                '''pos pair'''
                for hypo in hypo_list:
                    guid = "test-"+str(exam_co)
                    text_a = line[1]
                    text_b = hypo
                    label = 'entailment' #if line[0] == '1' else 'not_entailment'
                    examples.append(
                        InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
                    exam_co+=1
            else:
                '''neg pair'''
                for hypo in hypo_list:
                    guid = "test-"+str(exam_co)
                    text_a = line[1]
                    text_b = hypo
                    label = 'not_entailment' #if line[0] == '1' else 'not_entailment'
                    examples.append(
                        InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
                    exam_co+=1
        line_co+=1
        if line_co % 1000 == 0:
            print('loading test size:', line_co)

loading test size: 1000
loading test size: 2000
loading test size: 3000
loading test size: 4000
loading test size: 5000
loading test size: 6000
loading test size: 7000
loading test size: 8000
loading test size: 9000
loading test size: 10000
loading test size: 11000
loading test size: 12000
loading test size: 13000
loading test size: 14000
loading test size: 15000
loading test size: 16000
loading test size: 17000
loading test size: 18000
loading test size: 19000
loading test size: 20000
loading test size: 21000
loading test size: 22000
loading test size: 23000
loading test size: 24000
loading test size: 25000
loading test size: 26000
loading test size: 27000
loading test size: 28000
loading test size: 29000
loading test size: 30000
loading test size: 31000
loading test size: 32000
loading test size: 33000
loading test size: 34000
loading test size: 35000
loading test size: 36000
loading test size: 37000
loading test size: 38000
loading test size: 39000
loading test size: 40000
loading t

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('valhalla/bart-large-sst2')
ex = tokenizer(
    ['abc ab', 'abd dasdf'],
    ['sadf as', 'asdf asd'],
    max_length= 20,
    padding= 'max_length',
    truncation= True)

Downloading: 100%|██████████| 1.12k/1.12k [00:00<00:00, 865kB/s]
Downloading: 100%|██████████| 899k/899k [00:05<00:00, 170kB/s]  
Downloading: 100%|██████████| 456k/456k [00:03<00:00, 132kB/s]  
Downloading: 100%|██████████| 772/772 [00:00<00:00, 384kB/s]


In [17]:
tokenizer.batch_decode(ex['input_ids'])

['<s>abc ab</s></s>sadf as</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s>abd dasdf</s></s>asdf asd</s><pad><pad><pad><pad><pad><pad><pad>']

In [17]:
import numpy as np
x = np.array([1, 2, 3, 4, 5])
for i in x:
    print(i in [1,2])


True
True
False
False
False


In [4]:
!python main.py --debuging=True --batch_size=8

Global seed set to 42
Namespace(conda_env='ligning', model_checkpoint='facebook/bart-base', debuging=True, batch_size=16, data_train_path='', logger=True, enable_checkpointing=True, default_root_dir=None, gradient_clip_val=None, gradient_clip_algorithm=None, num_nodes=1, num_processes=None, devices=None, gpus=None, auto_select_gpus=False, tpu_cores=None, ipus=None, enable_progress_bar=True, overfit_batches=0.0, track_grad_norm=-1, check_val_every_n_epoch=1, fast_dev_run=False, accumulate_grad_batches=None, max_epochs=None, min_epochs=None, max_steps=-1, min_steps=None, max_time=None, limit_train_batches=None, limit_val_batches=None, limit_test_batches=None, limit_predict_batches=None, val_check_interval=None, log_every_n_steps=50, accelerator=None, strategy=None, sync_batchnorm=False, precision=32, enable_model_summary=True, weights_save_path=None, num_sanity_val_steps=2, resume_from_checkpoint=None, profiler=None, benchmark=None, deterministic=None, reload_dataloaders_every_n_epochs=0